# Install 

In [13]:
! pip install transformers datasets evaluate
! pip install sacrebleu
! pip install googletrans==3.1.0a0
! pip install huggingface_hub

In [12]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [4]:
from datasets import load_dataset, DatasetDict, Dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import pipeline
from googletrans import Translator

# Loading Dataset

In [ ]:
dataset = load_dataset("aslg_pc12")
dataset = dataset["train"].train_test_split(train_size=0.8)

# Back Translation

In [ ]:
dataset = dataset["train"].train_test_split(train_size=0.8)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

source_lang = "text"
target_lang = "gloss"


def preprocess_function_backtrans(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs,  text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_data = dataset.map(preprocess_function_backtrans, batched=True)


In [ ]:
sacrebleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("fuyulinh04/transformer_model")

model = AutoModelForSeq2SeqLM.from_pretrained("fuyulinh04/transformer_model")

In [23]:
translator = pipeline("translation", model=model, tokenizer = tokenizer)

prediction = []
answer = []

def testing_data():
  for i in range(7):
    text = dataset["train"][i]["text"]
    answer.append(dataset["train"][i]["gloss"])
    prediction.append(translator(text))

In [24]:
testing_data()
print(prediction)
print(answer)

[[{'translation_text': 'auf 25 MARS 1821 , GREECE BE NATION .'}], [{'translation_text': 'X-I MAKE THAT DESC-OFF FROM INTERNET X-I .'}], [{'translation_text': 'DESC-HOWEVER , THROUGH ALL DESC-TECHNOLOGICAL MEASURE , DESC-GREAT STAFF INDUSTRY .'}], [{'translation_text': 'DESC-RE BE COUNTRY THAT DO DESC-NOT HAVE ENERGY MARKET , DESC-AS SARAIN IN CASE GAS .'}], [{'translation_text': 'THIS BE PREISE EURO X-POSS FORCE SUCCESS .'}], [{'translation_text': 'FR MADAM PRESIDENT , EUROPE X-POSS GOVERNANCE BE AT SPEAKE WITH REGARD TO ENERGY ISSUE .'}], [{'translation_text': 'COMMISSION , COMMISSION , WILL BE DESC-DECISIONAL FOR EUROPEAN UNION Architecture FUTURE .'}]]
['ON 25 MARCH 1821 , GREECE BE BEAR .\n', 'X-I COULD HAVE PRINT THAT OFF FROM INTERNET X-MYSELF .\n', 'DESC-HOWEVER , DESPITE ALL DESC-TECHNICAL MEASURE , DESC-LARGE SCALE ACCIDENT CAN OCCUR .\n', 'DESC-RE BE COUNTRY THAT DO DESC-NOT HAVE ENERGY MARKET , DESC-SUCH AS SPAIN IN CASE GAS .\n', 'THIS BE PRICE EURO X-POSS CONTINUE SUCCESS 

# Training Model

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [6]:
source_lang = "gloss"
target_lang = "text"


def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs,  text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [7]:
tokenized_data = dataset.map(preprocess_function, batched=True)

  0%|          | 0/71 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

In [8]:
sacrebleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()

# Load Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("junowhite/transformer_model")

model = AutoModelForSeq2SeqLM.from_pretrained("junowhite/transformer_model")

In [ ]:
translators = Translator()

In [ ]:
translator = pipeline("translation", model=model, tokenizer = tokenizer)

prediction = []
answer = []

def testing_data():
  for i in range(7):
    text = dataset["train"][i]["gloss"]
    answer.append(dataset["train"][i]["text"])
    prediction.append(translators.translate(translator(text)[0]["translation_text"]).text)

In [ ]:
testing_data()
print(prediction)
print(answer)